In [1]:
import sys
sys.path.append('..')
from core import computation_graph
from core import graph_merge
from visualize_graph import network_graph
from image_utilities import load_images

import run_image_experiment

%load_ext autoreload
%autoreload 2

/home/jakeval/.local/pipx/venvs/notebook/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Overlapping Accounts

In [37]:
def _add_names_for_debugging(pair, x, y, incremented_x, incremented_y, incremented_pair):
    pair.name = "(2, 6)"
    x.name = "2"
    y.name = "6"
    incremented_x.name = "3"
    incremented_y.name = "7"
    incremented_pair.name = "(3, 7)"



    
    


@computation_graph.optex_process(['pair_1', 'pair_2'])
def split(pair):
    return pair[0], pair[1]


@computation_graph.optex_process('x_plus_1')
def increment(x):
    return x + 1


@computation_graph.optex_process('pair')
def join(pair_1, pair_2):
    return (pair_1, pair_2)


@computation_graph.optex_composition('pair + 1')
def add1ToAll(pair):

    x, y = split(pair)
    
    incremented_x = increment(x)
    
    incremented_y = increment(y)
    
    incremented_pair = join(incremented_x, incremented_y)
    
    _add_names_for_debugging(pair, x, y, incremented_x, incremented_y, incremented_pair)
    
    return incremented_pair





simple_graph = computation_graph.EdgeGraph.from_process(add1ToAll, "simple_graph")
network_graph.Network_Graph(simple_graph, show_agents=False, notebook=True).save_graph('test.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [38]:
expanded_copy = graph_merge.make_expanded_graph_copy(simple_graph)
network_graph.Network_Graph(expanded_copy, show_agents=False, notebook=True).save_graph('test.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


# Full Graph Merging

In [44]:
def _add_names_for_debugging(arg1, arg2, out_1, out_2, out, final):
    arg1.name = "arg1"
    arg2.name = "arg2"
    out_1.name = 'scale_out_1'
    out_2.name = 'scale_out_2'
    out.name = 'add_three_out'
    final.name = 'sum_out'




@computation_graph.optex_process(('scale_2_out', 'scale_4_out'))
def scale(arg1):
    return arg1*2, arg1*4

@computation_graph.optex_process('add_three_out')
def add_three(arg):
    return arg + 3

@computation_graph.optex_process('sum_output')
def sum_inputs(arg1, arg2):
    return arg1 + arg2


@computation_graph.optex_composition(('output_1', 'output_2'))
def dual_return(arg1, arg2):
    out_1, out_2 = scale(arg1)
    out = add_three(arg2)
    final = sum_inputs(out_2, out)
    
    _add_names_for_debugging(arg1, arg2, out_1, out_2, out, final)
    
    return out_1, final


graph1 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_return, "Graph 1"))
graph2 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_return, "Graph 2"))

unmerged_graph = computation_graph.EdgeGraph.from_output_artifacts(graph1.outputs + graph2.outputs, name='unmerged_graph')
network_graph.Network_Graph(unmerged_graph, notebook=True).save_graph('test.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [46]:
graph1 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_return, "Graph 1"))
graph2 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_return, "Graph 2"))

arg1 = computation_graph.Artifact(10)
arg1.name = 'arg1'
arg2 = computation_graph.Artifact(20)
arg2.name = 'arg2'
inputs = graph_merge.get_inputs([
    (graph1, scale, 'arg1', arg1),
    (graph1, add_three, 'arg', arg2),
    (graph2, scale, 'arg1', arg1),
    (graph2, add_three, 'arg', arg2)])

merged_graph, merged_inputs, merged_outputs = graph_merge.merge_graphs([graph1, graph2], inputs, "merged")
network_graph.Network_Graph(merged_graph, notebook=True).save_graph('test.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


# Partial Graph Merging

If only some of the arguments two each graph are identical, the graph can only be partially merged.

In [47]:
graph1 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_return, "Graph 1"))
graph2 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_return, "Graph 2"))


arg1_merged = computation_graph.Artifact(10)
arg1_merged.name = 'arg1_merged'
arg2_graph1 = computation_graph.Artifact(20)
arg2_graph1.name = 'arg2_graph1'
arg2_graph2 = computation_graph.Artifact(30)
arg2_graph2.name = 'arg2_graph2'

inputs = graph_merge.get_inputs([
    (graph1, scale, 'arg1', arg1_merged),
    (graph1, add_three, 'arg', arg2_graph1),
    (graph2, scale, 'arg1', arg1_merged),
    (graph2, add_three, 'arg', arg2_graph2)])

merged_graph, merged_inputs, merged_outputs = graph_merge.merge_graphs([graph1, graph2], inputs, "merged")
network_graph.Network_Graph(merged_graph, notebook=True).pyvis_graph.show('test.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
